# **Examples from https://doi.org/10.1016/j.aam.2020.102065** 
$\newcommand{\kdv}{\text{kdv}}\newcommand{\KdV}{\mathbf{\text{KdV}}}$
In this notebook we will simulate the computations within the article https://doi.org/10.1016/j.aam.2020.102065 using the software `dalgebra`. This will be useful as a demonstration on how to use the software, a testing on the capabilities of the software and a chance to explore future features for the software.

For each example we will include the following:
* Small introduction to the problem.
* A cell setting up the framework for the example.
* A sequence of cells and texts developing the computations of the example.
* A last sentence with the conclusion or how to interpret the result.

In [1]:
import sys; sys.path.insert(0, "..") # dalgebra is here -- comment for installed version

%display latex
from functools import lru_cache
from sage.categories.pushout import pushout
from sage.rings.polynomial.polynomial_ring import is_PolynomialRing
from sage.rings.polynomial.multi_polynomial_ring import is_MPolynomialRing

from dalgebra import *
from dalgebra.ring_w_operator import RingWithOperators_Wrapper, SkewMap
from dalgebra.rwo_polynomial.rwo_polynomial_element import RWOPolynomialGen

## **0. General functions**

In this section we include some cells with functionality that will be used later in the notebook.

This part can be considered to be pushed into a more general framework and added into the main code of the software.

In [2]:
def commutator(op1, op2, gen):
    r'''Return the commutator ``[op1,op2]``'''
    R = pushout(op1.parent(), op2.parent())
    op1 = R(op1); op2 = R(op2)
    if isinstance(gen, RWOPolynomialGen):
        gen = gen.variable_name()
    elif gen in ZZ:
        gen = R.variable_names()[gen]
    elif not isinstance(gen, str):
        raise TypeError("Generator must be a RWOPolynomialGen, a integer or a string with the name")

    if not gen in R.variable_names():
        raise ValueError("The given variable is not in the ring of the operators")
    
    return op1(**{gen : op2}) - op2(**{gen : op1})    

In [3]:
def __create_constants_as_polys(B, names):
    if not B in RingsWithOperators():
        raise TypeError("We require a ring with operators")
    if B.noperators() > 1:
        raise ValueError("We only allow one operator to be extended")
    if not isinstance(B, RingWithOperators_Wrapper):
        raise NotImplementedError("Only extending wrappers of known rings")
    old_d = B.operators()[0]
    ttype = B.operator_types()[0]
    if ttype == "none": raise ValueError("The given operator is not recognized --> impossible to detect constants")
    
    bB = B.wrapped
    if is_PolynomialRing(bB) or is_MPolynomialRing(bB):
        # We need to extend the variables and the derivation
        old_gens = [str(g) for g in bB.gens()]
        aux = PolynomialRing(bB, names); nB = aux.flattening_morphism().codomain()
        if ttype == "homomorphism":
            new_d = nB.Hom(nB)([nB(old_d.function(bB(str(g)))) if str(g) in old_gens else g for g in nB.gens()])
        elif ttype == "derivation":
            new_d = nB.derivation_module()(old_d.function.function.list() + [0 for _ in range(len(names))])
        elif ttype == "skew":
            new_d = nB.derivation_module(twist=old_d.function.twist)(old_d.function.function.list() + [0 for _ in range(len(names))])
        else:
            raise ValueError(f"Weird operator type detected ({ttype})")
    else: # we simply create a polynomial ring
        nB = PolynomialRing(bB, names)
        if ttype == "homomorphism":
            new_d = nB.Hom(nB).one()
        elif ttype == "derivation":
            new_d = nB.derivation_module().zero()
        elif ttype == "skew":
            new_d = nB.derivation_module(twist=old_d.function.twist).zero()
        else:
            raise ValueError(f"Weird operator type detected ({ttype})")
    
    return RingWithOperators(nB, new_d, types=[ttype])        

## **1. Section 3**

### The sequence $\kdv_n$

In the article, the $\kdv_n$ sequence is defined recursively using two pseudo-differential operators:

$$\mathcal{R} = -\frac{1}{4}\partial^2 + u + \frac{1}{2} u' \partial^{-1},\qquad \mathcal{R}^* = -\frac{1}{4}\partial^2 + u - \frac{1}{2}\partial^{-1} u,$$

where $u$ is a differential indeterminate from a differential field $(C,\partial)$. The inverse operator $\partial^{-1}$ returns an antiderivative of the input to the operator. This can pose a serious problem because each time we apply it, we obtain a new different constant. Let see if we can express this directly on the system:

In [4]:
#B = DifferentialRing(InfinitePolynomialRing(QQ, "c"), lambda p : 0)

Unfortunately, the ring of infinitely many variables polynomial ring do not have a derivation module implemented. We can consider to implement one ourselves at some point, or simply try and avoid this by creating explicit variables for the integration constants when needed.

In [5]:
B = DifferentialRing(QQ, lambda p : 0) # change QQ to other polynomial ring with more constants if needed
R.<u> = DifferentialPolynomialRing(B)

The $\kdv_n$ operators are dfeined recursively:
$$\kdv_{n+1} = \mathcal{R}(\kdv_n),\qquad \kdv_0 = u'.$$
On the other hand, we can also define another recurrence using the adjoint operator $\mathcal{R}^*$:
$$v_{n+1} = \mathcal{R}^*(v_n),\qquad v_0 = 1.$$

It is shown in Equation (6) that these two sequences are related (due to the fact that $\mathcal{R}$ and $\mathcal{R}^*$ are adjoints):
$$2v_{n+1}' = \kdv_n.$$

In the following cell, we include two methods: the first tries to check whether a polynomial in $C\{u\}$ is a total derivative using a recursive elimination approach. The second method computes the application of $\mathcal{R}^*$ for any element $p(u) \in C\{u\}$ for which we know $u'p(u)$ is a total derivative (i.e., we know of some $a(u) \in C\{u\}$ such that $a'(u) = u'p(u)$.

In [6]:
def is_total_derivative(p, gen = None):
    # Getting variables from argument
    original = p
    R = p.parent(); B = R.base()
    if R.noperators() > 1: raise ValueError(f"Too many operators ({R.noperators()})")
    if not R.is_differential(): raise TypeError("Only differential fields are allowed")
    u = R.gen(gen if gen != None else 0)
    
    a = 0
    while(p != 0):
        #print("**********************************")
        #print(f"** Remaining polynomial: {p}\n-- Current antiderivative: {a}")
        order = p.order(u)
        #print(f"** Current order: {order}")
        if p.degree(u[order]) > 1: return False, f"The degree of the highest derivative is too big ({p.degree(u[p.order(u)])})"
    
        ## extracting parts
        highest_order = p.coefficient(u[order]); deg_order_1 = highest_order.degree(u[order-1])
        cs = [highest_order.coefficient(u[order-1]**i) for i in range(deg_order_1+1)]
        order_1_part = sum(cs[i]*u[order-1]**i for i in range(deg_order_1+1))
        q1 = highest_order - order_1_part # order q1 < d-1
        
        #print(f"    Order = d-1: {order_1_part}")
        #print(f"    Order < d-1: {q1}")
        new_a = sum(cs[i]*(1/(i+1)) * u[order-1]**(i+1) for i in range(deg_order_1+1)) + u[order-1]*q1
        p -= new_a.derivative()
        a += new_a
        
    assert a.derivative() == original
    return True, a

def apply_Rs(p, a, gen=0):
    r'''a' = u'*p'''
    R = p.parent(); u = R.gen(gen)
    new_p = -(1/4)*p.derivative(times=2) + u[0]*p - (1/2)*a
    is_total, new_a = is_total_derivative(u[1]*new_p, gen)
    if is_total:
        return new_p, new_a
    else:
        raise ValueError(f"u'P is not a total derivative: {new_a}")    
    return new_p, new_a

Hence, we can now compute the $\kdv_n$ sequence:

In [7]:
def Rs(p,a,gen=0,times=1):
    if times == 1:
        return apply_Rs(p,a,gen)
    return Rs(*Rs(p,a,gen=gen,times=times-1),gen=gen)

@lru_cache(maxsize=None)
def v(n, u):
    return Rs(u._parent.one(), u[0], u._parent.gens().index(u), n)[0]

@lru_cache(maxsize=None)
def kdv(n, u):
    vn1 = v(n+1, u)
    return 2*vn1.derivative()

In [8]:
show(r"kdv(0) \longrightarrow " + latex(kdv(0,u)))
show(r"kdv(1) \longrightarrow " + latex(kdv(1,u)))
show(r"kdv(2) \longrightarrow " + latex(kdv(2,u)))
show(r"kdv(3) \longrightarrow " + latex(kdv(3,u)))

kdv(0) \longrightarrow 1 u_{1}

kdv(1) \longrightarrow \frac{3}{2} u_{1} u_{0} + \left(-\frac{1}{4}\right) u_{3}

kdv(2) \longrightarrow \frac{15}{8} u_{1} u_{0}^{2} + \left(-\frac{5}{8}\right) u_{3} u_{0} + \left(-\frac{5}{4}\right) u_{2} u_{1} + \frac{1}{16} u_{5}

kdv(3) \longrightarrow \frac{35}{16} u_{1} u_{0}^{3} + \left(-\frac{35}{32}\right) u_{3} u_{0}^{2} + \left(-\frac{35}{8}\right) u_{2} u_{1} u_{0} + \left(-\frac{35}{32}\right) u_{1}^{3} + \frac{7}{32} u_{5} u_{0} + \frac{21}{32} u_{4} u_{1} + \frac{35}{32} u_{3} u_{2} + \left(-\frac{1}{64}\right) u_{7}

### The Schrödinger operators

The $\kdv_n$ and $v_n$ sequences are useful to define some differential operators that will be related with the Schrödinger operator $L_u = L = -\partial^2 + u$. We see that to define this operator, we need two differential variables: $y$ will denote the solution to the operator and $u$ will be an arbitrary differential variable.

In [9]:
SchR.<u,y> = DifferentialPolynomialRing(QQ); show(SchR)
L = -y[2] + u[0]*y[0]; show(r"L \mapsto " + latex(L))

Ring of operator polynomials in (u, y) over Differential Ring [[Rational Field], (0,)]

L \mapsto 1 u_{0} y_{0} - y_{2}

For the operator $L_u$, we will define a sequence of odd order operators $P_{2n+1}$ that will commute with $L$. This sequence of operators are defined recursively using also the sequence $v_n$ and the operator $L$:

In [10]:
@lru_cache(maxsize=None)
def P_odd(n,u,y,L):
    r'''Computes P_{2n+1}'''
    if n == 0: # return $P_1$
        return y[1]
    else:
        vn = v(n,u)
        return vn*y[1] - (1/2)*vn.derivative()*y[0] + P_odd(n-1, u, y, L)(y=L)

In [11]:
show(r"P_1 \mapsto " + latex(P_odd(0,u,y,L)))
show(r"P_3 \mapsto " + latex(P_odd(1,u,y,L)))
show(r"P_5 \mapsto " + latex(P_odd(2,u,y,L)))
show(r"P_7 \mapsto " + latex(P_odd(3,u,y,L)))

P_1 \mapsto 1 y_{1}

P_3 \mapsto \frac{3}{4} u_{1} y_{0} + \frac{3}{2} u_{0} y_{1} - y_{3}

P_5 \mapsto \frac{15}{8} u_{1} u_{0} y_{0} + \frac{15}{8} u_{0}^{2} y_{1} + \left(-\frac{15}{16}\right) u_{3} y_{0} + \left(-\frac{25}{8}\right) u_{2} y_{1} + \left(-\frac{15}{4}\right) u_{1} y_{2} + \left(-\frac{5}{2}\right) u_{0} y_{3} + 1 y_{5}

P_7 \mapsto \frac{105}{32} u_{1} u_{0}^{2} y_{0} + \frac{35}{16} u_{0}^{3} y_{1} + \left(-\frac{105}{32}\right) u_{3} u_{0} y_{0} + \left(-\frac{105}{16}\right) u_{2} u_{1} y_{0} + \left(-\frac{175}{16}\right) u_{2} u_{0} y_{1} + \left(-\frac{245}{32}\right) u_{1}^{2} y_{1} + \left(-\frac{105}{8}\right) u_{1} u_{0} y_{2} + \left(-\frac{35}{8}\right) u_{0}^{2} y_{3} + \frac{63}{64} u_{5} y_{0} + \frac{161}{32} u_{4} y_{1} + \frac{175}{16} u_{3} y_{2} + \frac{105}{8} u_{2} y_{3} + \frac{35}{4} u_{1} y_{4} + \frac{7}{2} u_{0} y_{5} - y_{7}

It is here when the key property between the Schrödinger operator $L_u$ and the operators $P_{2n+1}$ comes into place (written in Lemma 3.2):
$$[P_{2n+1}, L] = \kdv_n$$

In [12]:
%%time
[
    (2*i+1,commutator(P_odd(i,u,y,L), L, y) == kdv(i,u)*y[0]) # we divide by y_0 to see clearly the kdv formula from before.
    for i in range(10)
]

CPU times: user 19.9 s, sys: 129 ms, total: 20 s
Wall time: 20 s


[(1, True),
 (3, True),
 (5, True),
 (7, True),
 (9, True),
 (11, True),
 (13, True),
 (15, True),
 (17, True),
 (19, True)]

### The $\KdV_n$ differential polynomials

Right after Lemma 3.2, a new (more complicated notion) is defined: the $\KdV_n$ differential polynomials are linear combinations of the $\kdv_n$ polynomials. Since the $\kdv_n$ differential polynomials have order $n$, then they form a basis (as polynomials) of the space of differential operators. The $\KdV_n$ sequence represents a recombination of this basis into a new one:
$$\KdV_0 = u',\qquad \KdV_n = \kdv_n + \sum_{l=0}^{n-1} c_{n-l}\kdv_l.$$

The constants $c_i$ are algebraically independent variables. We can see that, to produce these polynomials we need to know the value of $n$ in order to generate the constants $c_1,\ldots,c_n$. This makes things slightly mroe complicated, but the following method will provide a solution using the softare in `dalgebra`:

In [12]:
def KdV(n, u):
    r'''Produces KdV_n(u,c^n)'''
    R = u._parent # this is a ring of differential polynomials
    B = R.base() # this is a differential ring (in our case, a Wrapper)
    nB = __create_constants_as_polys(B, [f"c_{i+1}" for i in range(n)])
    cs = [nB(f"c_{i+1}") for i in range(n)]
    nR = R.change_ring(nB)
    nu = nR.gens()[nR.variable_names().index(u.variable_name())]

    return kdv(n, nu) + sum(cs[-(l+1)]*kdv(l,nu) for l in range(n))

In [13]:
KdV(3, u)

35/16*u_1*u_0^3 + (-35/32)*u_3*u_0^2 + (-35/8)*u_2*u_1*u_0 + (-35/32)*u_1^3 + 15/8*c_1*u_1*u_0^2 + 7/32*u_5*u_0 + 21/32*u_4*u_1 + 35/32*u_3*u_2 + (-5/8*c_1)*u_3*u_0 + (-5/4*c_1)*u_2*u_1 + 3/2*c_2*u_1*u_0 + (-1/64)*u_7 + 1/16*c_1*u_5 + (-1/4*c_2)*u_3 + c_3*u_1

Then, similarly to the Schrödinger odd order differential operators $P_{2n+1}$ defined above, we can define a new odd order differential operator involving the constants $c_i$ that appear in the $\KdV_n$ polynomials. A similar commutation with $L$ will happen again:

In [14]:
@lru_cache(maxsize=None)
def Phat_odd(n,u,y,L):
    r'''Computes \hat{P}_{2n+1}'''
    if n == 0: # return $P_1$
        return y[1]
    else:
        R = KdV(n,u).parent() # we create the corresponding constants
        u = R.gen(u._parent.gens().index(u)); y = R.gen(y._parent.gens().index(y)) # we transform the generators to the new ring
        L = R(L) # we transform the Schrodinger operator to the new ring
        cs = [R.base()(f"c_{i+1}") for i in range(n)]
        
        return P_odd(n,u,y,L) + sum(cs[-(l+1)]*P_odd(l, u, y, L) for l in range(n))

In [16]:
%%time
[
    (2*i+1,commutator(Phat_odd(i,u,y,L), L, y) == KdV(i, u)*y[0]) # we divide by y_0 to see clearly the kdv formula from before.
    for i in range(10)
]


CPU times: user 25.8 s, sys: 570 ms, total: 26.4 s
Wall time: 26.4 s


[(1, True),
 (3, True),
 (5, True),
 (7, True),
 (9, True),
 (11, True),
 (13, False),
 (15, False),
 (17, False),
 (19, False)]

This <font color='red'>seems to be a problem</font> for cases over 13!!

### Example 3.3: Differential Sylvester resultant

In Example 3.3 of the article, a generic resultant is computed for two differential polynomials. We can reproduce that computation with this software. In fact, we can distinguish between four cases: 
1. The generic coefficients are constants
2. The generic coefficients are elementary functions
3. The generic coefficients are linked by the derivation
4. The generic coefficients are fully independent

#### Case 1: coefficients are constants

In [4]:
B = DifferentialRing(QQ["a_0,a_1,a_2,b_0,b_1,b_2,b_3"], lambda p : 0)
T.<d> = DifferentialPolynomialRing(B)
a_0,a_1,a_2,b_0,b_1,b_2,b_3 = T.base().gens()

P1 = a_2*d[2] + a_1*d[1] + a_0*d[0]; P2 = b_3*d[3] + b_2*d[2] + b_1*d[1] + b_0*d[0]

In [5]:
%%time
resultant_case1 = P1.sylvester_resultant(P2, d)
resultant_case1

CPU times: user 92.1 ms, sys: 0 ns, total: 92.1 ms
Wall time: 91.6 ms


a_2^3*b_0^2 - a_1*a_2^2*b_0*b_1 + a_0*a_2^2*b_1^2 + a_1^2*a_2*b_0*b_2 - 2*a_0*a_2^2*b_0*b_2 - a_0*a_1*a_2*b_1*b_2 + a_0^2*a_2*b_2^2 - a_1^3*b_0*b_3 + 3*a_0*a_1*a_2*b_0*b_3 + a_0*a_1^2*b_1*b_3 - 2*a_0^2*a_2*b_1*b_3 - a_0^2*a_1*b_2*b_3 + a_0^3*b_3^2

#### Case 2: coefficients are elementary functions

Let assume that we know some differential behavior of the coefficients. Namely:
$$\left\{\begin{array}{rll}
    a_0' = & 1,&\longrightarrow a_0 = x\\
    a_1' = & \frac{1}{a_0+1}, &\longrightarrow a_1 = \log(x+1)\\
    a_2' = & a_2, &\longrightarrow a_2 = e^x\\
    b_0' = & 2a_0b_0, &\longrightarrow b_0 = e^{x^2}\\
    b_1' = & \frac{2a_0}{a_0^2 + 1}, &\longrightarrow b_1 = \log(x^2 + 1)\\
    b_2' = & 0, &\longrightarrow b_2 = c\\
    b_3' = & a_2b_3 &\longrightarrow b_3 = e^{e^{x}}
\end{array}\right.$$

To create this ring we need first to define the derivation on the ground field and then build the differential extension to build the operators:

In [6]:
## Building the base differential field
B.<a_0,a_1,a_2,b_0,b_1,b_2,b_3> = QQ[]
F = B.fraction_field()
da_0,da_1,da_2,db_0,db_1,db_2,db_3 = F.derivation_module().gens()
D = da_0 + (1/(a_0+1))*da_1 + a_2*da_2 + 2*a_0*b_0*db_0 + 2*a_0/(a_0^2+1)*db_1 + a_2*b_3*db_3

## Creating the ring for differential operators
T.<d> = DifferentialPolynomialRing(DifferentialRing(F, D))

## Creating the two differential operators
P1 = a_2*d[2] + a_1*d[1] + a_0*d[0]; P2 = b_3*d[3] + b_2*d[2] + b_1*d[1] + b_0*d[0]

In [7]:
%%time
resultant_case2 = P1.sylvester_resultant(P2, d)
resultant_case2

CPU times: user 150 ms, sys: 9.98 ms, total: 160 ms
Wall time: 159 ms


(-2*a_0^5*a_2^3*b_0*b_1 + 2*a_0^5*a_1*a_2^2*b_0*b_2 - 2*a_0^5*a_1^2*a_2*b_0*b_3 + 2*a_0^6*a_2^2*b_0*b_3 - 2*a_0^5*a_1*a_2^2*b_0*b_3 + a_0^4*a_1^2*a_2^2*b_0*b_3 - a_0^5*a_2^3*b_0*b_3 + a_0^4*a_1*a_2^3*b_0*b_3 - a_0^5*a_1*a_2^2*b_1*b_3 - a_0^5*a_2^3*b_1*b_3 + a_0^6*a_2^2*b_2*b_3 + a_0^4*a_2^3*b_0^2 - a_0^4*a_1*a_2^2*b_0*b_1 - 4*a_0^4*a_2^3*b_0*b_1 + a_0^5*a_2^2*b_1^2 + a_0^4*a_1^2*a_2*b_0*b_2 - 2*a_0^5*a_2^2*b_0*b_2 + 5*a_0^4*a_1*a_2^2*b_0*b_2 - a_0^5*a_1*a_2*b_1*b_2 - a_0^5*a_2^2*b_1*b_2 + a_0^6*a_2*b_2^2 - a_0^4*a_1^3*b_0*b_3 + 3*a_0^5*a_1*a_2*b_0*b_3 - 7*a_0^4*a_1^2*a_2*b_0*b_3 + 7*a_0^5*a_2^2*b_0*b_3 - 5*a_0^4*a_1*a_2^2*b_0*b_3 + 2*a_0^3*a_1^2*a_2^2*b_0*b_3 - 2*a_0^4*a_2^3*b_0*b_3 + 2*a_0^3*a_1*a_2^3*b_0*b_3 + a_0^5*a_1^2*b_1*b_3 - 2*a_0^6*a_2*b_1*b_3 + 3*a_0^5*a_1*a_2*b_1*b_3 + a_0^5*a_2^2*b_1*b_3 - 2*a_0^4*a_1*a_2^2*b_1*b_3 - a_0^4*a_2^3*b_1*b_3 - a_0^6*a_1*b_2*b_3 - 2*a_0^6*a_2*b_2*b_3 + 2*a_0^5*a_2^2*b_2*b_3 - a_0^4*a_1*a_2^2*b_2*b_3 + a_0^7*b_3^2 + a_0^6*a_2*b_3^2 + 2*a_0^3*a_2^3*b_0^2 - 2*a_0^3*a_1*a_2^2*b_0*b_1 - 4*a_0^3*a_2^3*b_0*b_1 + 2*a_0^4*a_2^2*b_1^2 + 2*a_0^3*a_1^2*a_2*b_0*b_2 - 4*a_0^4*a_2^2*b_0*b_2 + 6*a_0^3*a_1*a_2^2*b_0*b_2 - 2*a_0^4*a_1*a_2*b_1*b_2 - a_0^4*a_2^2*b_1*b_2 + 2*a_0^5*a_2*b_2^2 - a_0^4*a_1*a_2*b_2^2 - 2*a_0^3*a_1^3*b_0*b_3 + 6*a_0^4*a_1*a_2*b_0*b_3 - 10*a_0^3*a_1^2*a_2*b_0*b_3 + 9*a_0^4*a_2^2*b_0*b_3 - 6*a_0^3*a_1*a_2^2*b_0*b_3 + 2*a_0^2*a_1^2*a_2^2*b_0*b_3 - 3*a_0^3*a_2^3*b_0*b_3 + 2*a_0^2*a_1*a_2^3*b_0*b_3 + 2*a_0^4*a_1^2*b_1*b_3 - 4*a_0^5*a_2*b_1*b_3 + 6*a_0^4*a_1*a_2*b_1*b_3 - 2*a_0^3*a_1*a_2^2*b_1*b_3 - 2*a_0^5*a_1*b_2*b_3 + a_0^4*a_1^2*b_2*b_3 - 2*a_0^5*a_2*b_2*b_3 + 2*a_0^4*a_1*a_2*b_2*b_3 + 3*a_0^4*a_2^2*b_2*b_3 - 2*a_0^3*a_1*a_2^2*b_2*b_3 + 2*a_0^6*b_3^2 - 3*a_0^5*a_1*b_3^2 - a_0^4*a_1*a_2*b_3^2 + 2*a_0^3*a_2^3*b_0 + 2*a_0^2*a_2^3*b_0^2 - 2*a_0^2*a_1*a_2^2*b_0*b_1 - 4*a_0^2*a_2^3*b_0*b_1 + 2*a_0^3*a_2^2*b_1^2 - 2*a_0^4*a_2^2*b_2 + 2*a_0^2*a_1^2*a_2*b_0*b_2 - 5*a_0^3*a_2^2*b_0*b_2 + 6*a_0^2*a_1*a_2^2*b_0*b_2 - 2*a_0^3*a_1*a_2*b_1*b_2 + 3*a_0^4*a_2*b_2^2 - 2*a_0^3*a_1*a_2*b_2^2 + 2*a_0^4*a_1*a_2*b_3 + 2*a_0^4*a_2^2*b_3 - 2*a_0^2*a_1^3*b_0*b_3 + 9*a_0^3*a_1*a_2*b_0*b_3 - 10*a_0^2*a_1^2*a_2*b_0*b_3 + 8*a_0^3*a_2^2*b_0*b_3 - 6*a_0^2*a_1*a_2^2*b_0*b_3 + 2*a_0*a_1^2*a_2^2*b_0*b_3 - 3*a_0^2*a_2^3*b_0*b_3 + 2*a_0*a_1*a_2^3*b_0*b_3 + 2*a_0^3*a_1^2*b_1*b_3 - 7*a_0^4*a_2*b_1*b_3 + 6*a_0^3*a_1*a_2*b_1*b_3 - 2*a_0^3*a_2^2*b_1*b_3 - 2*a_0^2*a_1*a_2^2*b_1*b_3 - 3*a_0^4*a_1*b_2*b_3 + 2*a_0^3*a_1^2*b_2*b_3 - 2*a_0^4*a_2*b_2*b_3 + 4*a_0^3*a_1*a_2*b_2*b_3 + 3*a_0^3*a_2^2*b_2*b_3 - 2*a_0^2*a_1*a_2^2*b_2*b_3 + 5*a_0^5*b_3^2 - 6*a_0^4*a_1*b_3^2 + a_0^4*a_2*b_3^2 - 2*a_0^3*a_1*a_2*b_3^2 + 4*a_0^2*a_2^3*b_0 + 2*a_0*a_2^3*b_0^2 - 2*a_0*a_1*a_2^2*b_0*b_1 - 2*a_0*a_2^3*b_0*b_1 + 2*a_0^2*a_2^2*b_1^2 - 4*a_0^3*a_2^2*b_2 + 2*a_0*a_1^2*a_2*b_0*b_2 - 5*a_0^2*a_2^2*b_0*b_2 + 4*a_0*a_1*a_2^2*b_0*b_2 - 2*a_0^2*a_1*a_2*b_1*b_2 + 3*a_0^3*a_2*b_2^2 - 2*a_0^2*a_1*a_2*b_2^2 + 4*a_0^3*a_1*a_2*b_3 + 2*a_0^3*a_2^2*b_3 - 2*a_0*a_1^3*b_0*b_3 + 9*a_0^2*a_1*a_2*b_0*b_3 - 8*a_0*a_1^2*a_2*b_0*b_3 + 7*a_0^2*a_2^2*b_0*b_3 - 4*a_0*a_1*a_2^2*b_0*b_3 + a_1^2*a_2^2*b_0*b_3 - 2*a_0*a_2^3*b_0*b_3 + a_1*a_2^3*b_0*b_3 + 2*a_0^2*a_1^2*b_1*b_3 - 7*a_0^3*a_2*b_1*b_3 + 6*a_0^2*a_1*a_2*b_1*b_3 - 2*a_0^2*a_2^2*b_1*b_3 - a_0*a_1*a_2^2*b_1*b_3 + a_0*a_2^3*b_1*b_3 - 3*a_0^3*a_1*b_2*b_3 + 2*a_0^2*a_1^2*b_2*b_3 - 3*a_0^3*a_2*b_2*b_3 + 4*a_0^2*a_1*a_2*b_2*b_3 + 2*a_0^2*a_2^2*b_2*b_3 - 2*a_0*a_1*a_2^2*b_2*b_3 + 5*a_0^4*b_3^2 - 7*a_0^3*a_1*b_3^2 + 4*a_0^3*a_2*b_3^2 - 2*a_0^2*a_1*a_2*b_3^2 + 2*a_0*a_2^3*b_0 + a_2^3*b_0^2 - a_1*a_2^2*b_0*b_1 + a_0*a_2^2*b_1^2 - 2*a_0^2*a_2^2*b_2 + a_1^2*a_2*b_0*b_2 - 3*a_0*a_2^2*b_0*b_2 + a_1*a_2^2*b_0*b_2 - a_0*a_1*a_2*b_1*b_2 + a_0*a_2^2*b_1*b_2 + 2*a_0^2*a_2*b_2^2 - 2*a_0*a_1*a_2*b_2^2 + 2*a_0^2*a_1*a_2*b_3 - 2*a_0^2*a_2^2*b_3 - a_1^3*b_0*b_3 + 6*a_0*a_1*a_2*b_0*b_3 - 3*a_1^2*a_2*b_0*b_3 + a_0*a_2^2*b_0*b_3 - a_1*a_2^2*b_0*b_3 - a_2^3*b_0*b_3 + a_0*a_1^2*b_1*b_3 - 5*a_0^2*a_2*b_1*b_3 + 3*a_0*a_1*a_2*b_1*b_3 - 3*a_0*a_2^2*b_

#### Case 3: coefficients are linked by the derivation

In this case, we are going to build the following differential ring:

$$a_0' = a_1,\quad a_1' = a_2,\quad a_2' = a_0.$$
$$b_0' = b_1,\quad b_1' = b_2,\quad b_2' = b_3,\quad b_3' = b_0.$$

In [8]:
## Building the base differential field
B.<a_0,a_1,a_2,b_0,b_1,b_2,b_3> = QQ[]
da_0,da_1,da_2,db_0,db_1,db_2,db_3 = B.derivation_module().gens()
D = a_1*da_0 + a_2*da_1 + a_0*da_2 + b_1*db_0 + b_2*db_1 + b_3*db_2 + b_0*db_3

## Creating the ring for differential operators
T.<d> = DifferentialPolynomialRing(DifferentialRing(F, D))

## Creating the two differential operators
P1 = a_2*d[2] + a_1*d[1] + a_0*d[0]; P2 = b_3*d[3] + b_2*d[2] + b_1*d[1] + b_0*d[0]

In [9]:
%%time
resultant_case3 = P1.sylvester_resultant(P2, d)
resultant_case3

CPU times: user 113 ms, sys: 184 µs, total: 113 ms
Wall time: 113 ms


a_0*a_1*a_2*b_0^2 + a_1^2*a_2*b_0^2 - a_0*a_2^2*b_0^2 - a_0^2*a_2*b_0*b_1 - a_0*a_1*a_2*b_0*b_1 + a_0*a_2^2*b_1^2 - a_2^3*b_1^2 + a_0^2*a_2*b_0*b_2 + a_0*a_1*a_2*b_0*b_2 - a_0*a_2^2*b_0*b_2 - a_0^2*a_2*b_1*b_2 - a_0*a_1*a_2*b_1*b_2 + 2*a_1*a_2^2*b_1*b_2 + a_0^2*a_2*b_2^2 - a_1^2*a_2*b_2^2 - 2*a_0^2*a_1*b_0*b_3 - 3*a_0*a_1^2*b_0*b_3 - a_1^3*b_0*b_3 + 3*a_0^2*a_2*b_0*b_3 + 3*a_0*a_1*a_2*b_0*b_3 + a_1^2*a_2*b_0*b_3 + a_0*a_2^2*b_0*b_3 - a_1*a_2^2*b_0*b_3 + 2*a_0^3*b_1*b_3 + 3*a_0^2*a_1*b_1*b_3 + a_0*a_1^2*b_1*b_3 - 2*a_0^2*a_2*b_1*b_3 - 4*a_0*a_1*a_2*b_1*b_3 - a_1^2*a_2*b_1*b_3 - a_0*a_2^2*b_1*b_3 + 2*a_2^3*b_1*b_3 - 2*a_0^3*b_2*b_3 - a_0^2*a_1*b_2*b_3 + 2*a_0*a_1^2*b_2*b_3 + a_1^3*b_2*b_3 + 2*a_0*a_1*a_2*b_2*b_3 - 2*a_1*a_2^2*b_2*b_3 - 2*a_0^2*a_1*b_3^2 - 3*a_0*a_1^2*b_3^2 - a_1^3*b_3^2 + 2*a_0^2*a_2*b_3^2 + 3*a_0*a_1*a_2*b_3^2 + 2*a_1^2*a_2*b_3^2 - a_2^3*b_3^2

#### Case 4: coefficients are fully independent

In this case, since we do not have any information about the derivations of the coefficients, they have to be included in the final ring of differential polynomials.

In [10]:
T.<a0,a1,a2,b0,b1,b2,b3,d> = DifferentialPolynomialRing(QQ)

## Creating the two differential operators
P1 = a2[0]*d[2] + a1[0]*d[1] + a0[0]*d[0]; P2 = b3[0]*d[3] + b2[0]*d[2] + b1[0]*d[1] + b0[0]*d[0]

In [11]:
%%time
resultant_case4 = P1.sylvester_resultant(P2, d)
print(resultant_case4) # too long for MathJax to process

-a0_2*a0_0*a2_0*b3_0^2 - a0_2*a1_1*a2_0*b3_0^2 + a0_2*a1_0^2*b3_0^2 + a0_2*a1_0*a2_1*b3_0^2 - a0_2*a1_0*a2_0*b2_0*b3_0 + a0_2*a2_0^2*b1_0*b3_0 + 2*a0_1^2*a2_0*b3_0^2 + (-3)*a0_1*a0_0*a1_0*b3_0^2 + (-2)*a0_1*a0_0*a2_1*b3_0^2 + 2*a0_1*a0_0*a2_0*b2_0*b3_0 + a0_1*a1_2*a2_0*b3_0^2 + (-2)*a0_1*a1_1*a1_0*b3_0^2 + a0_1*a1_0^2*b2_0*b3_0 - a0_1*a1_0*a2_2*b3_0^2 + 2*a0_1*a1_0*a2_1*b2_0*b3_0 + a0_1*a1_0*a2_0*b2_1*b3_0 - a0_1*a1_0*a2_0*b2_0^2 - a0_1*a1_0*a2_0*b2_0*b3_1 + (-2)*a0_1*a2_1*a2_0*b1_0*b3_0 + (-3)*a0_1*a2_0^2*b0_0*b3_0 - a0_1*a2_0^2*b1_1*b3_0 + a0_1*a2_0^2*b1_0*b2_0 + a0_1*a2_0^2*b1_0*b3_1 + a0_0^3*b3_0^2 + 3*a0_0^2*a1_1*b3_0^2 - a0_0^2*a1_0*b2_0*b3_0 + a0_0^2*a2_2*b3_0^2 + (-2)*a0_0^2*a2_1*b2_0*b3_0 + (-2)*a0_0^2*a2_0*b1_0*b3_0 - a0_0^2*a2_0*b2_1*b3_0 + a0_0^2*a2_0*b2_0^2 + a0_0^2*a2_0*b2_0*b3_1 - a0_0*a1_2*a1_0*b3_0^2 - a0_0*a1_2*a2_1*b3_0^2 + a0_0*a1_2*a2_0*b2_0*b3_0 + 2*a0_0*a1_1^2*b3_0^2 - a0_0*a1_1*a1_0*b2_0*b3_0 + a0_0*a1_1*a2_2*b3_0^2 + (-2)*a0_0*a1_1*a2_1*b2_0*b3_0 + (-3)*a0_0*a1

In [12]:
l = max(len(str(m)) for m in resultant_case4.monomials())
for c,m in zip(resultant_case4.coefficients(), resultant_case4.monomials()):
    print(f"{m}".ljust(l+2, " ") + f" --> {c}")

a0_2*a0_0*a2_0*b3_0^2      --> -1
a0_2*a1_1*a2_0*b3_0^2      --> -1
a0_2*a1_0^2*b3_0^2         --> 1
a0_2*a1_0*a2_1*b3_0^2      --> 1
a0_2*a1_0*a2_0*b2_0*b3_0   --> -1
a0_2*a2_0^2*b1_0*b3_0      --> 1
a0_1^2*a2_0*b3_0^2         --> 2
a0_1*a0_0*a1_0*b3_0^2      --> -3
a0_1*a0_0*a2_1*b3_0^2      --> -2
a0_1*a0_0*a2_0*b2_0*b3_0   --> 2
a0_1*a1_2*a2_0*b3_0^2      --> 1
a0_1*a1_1*a1_0*b3_0^2      --> -2
a0_1*a1_0^2*b2_0*b3_0      --> 1
a0_1*a1_0*a2_2*b3_0^2      --> -1
a0_1*a1_0*a2_1*b2_0*b3_0   --> 2
a0_1*a1_0*a2_0*b2_1*b3_0   --> 1
a0_1*a1_0*a2_0*b2_0^2      --> -1
a0_1*a1_0*a2_0*b2_0*b3_1   --> -1
a0_1*a2_1*a2_0*b1_0*b3_0   --> -2
a0_1*a2_0^2*b0_0*b3_0      --> -3
a0_1*a2_0^2*b1_1*b3_0      --> -1
a0_1*a2_0^2*b1_0*b2_0      --> 1
a0_1*a2_0^2*b1_0*b3_1      --> 1
a0_0^3*b3_0^2              --> 1
a0_0^2*a1_1*b3_0^2         --> 3
a0_0^2*a1_0*b2_0*b3_0      --> -1
a0_0^2*a2_2*b3_0^2         --> 1
a0_0^2*a2_1*b2_0*b3_0      --> -2
a0_0^2*a2_0*b1_0*b3_0      --> -2
a0_0^2*a2_0*b2_1*b3_0      -

### Subresultant sequences

### Subresultant sequences

Following the notation in subsection 3.2.2 of the paper, we will define the subresultant sequence for two linear operators: let $P, Q$ be two linear operators with orders $n$ and $m$ respectively. For $k \in \{0,\ldots,N\}$ where $N = \min(n,m)-1$, we define the matrix $S_k(P,Q)$ as the coefficient matrix (w.r.t. the main operator) of the operators 
$$\Xi_k(P,Q) = \{P,\partial P\ldots, \partial^{m-1-k} Q, Q, \partial Q,\ldots, \partial^{n-1-k} Q\}.$$

The matrix $S_k(P,Q)$ has always $n+m-2k$ rows and $n+m-k$ columns. In particular, when $k = 0$, we have the Sylvester matrix leading the the Sylvester resultant. Hence, we can see the matrices $S_k(P,Q)$ as specific submatrices of $S(P,Q) = S_0(P,Q)$ where we have removed the higher order rows and the columns that are fully zero.

**The computation of this matrix is included in the method `sylvester_matrix` by providing an extra argument for $k$.**

As remarked before, the matrices $S_k(P,Q)$ have $k$ more columns than rows. In order to obtain a square matrix, we need to remove $k$ columns. We can do that by removing $k$ out of the $k+1$ columns of $S_k(P,Q)$, or equivalently, $k$ columns from those indexed by ${1,\ldots,\partial^k}$. The matrix that keeps the partix for $\partial^i$ is denoted with $S_k^i(P,Q)$. 

**The determinants of these matrices can be obtained with the method `sylvester_subresultant` by givin both $k$ and $i$.**

Then, observe that these subresultants do not have the operator $\partial$. Hence, we can build new linear operators using these subresultants:
$$\mathcal{L}_k = \sum_{i=0}^k \det(S_k^i(P,Q))\partial^i.$$
The sequence $[\mathcal{L}_0,\ldots, \mathcal{L}_N]$ is called *subresultant sequence* and, as shown in the paper, it has nice properties related with the *gcd* of the operators $P$ and $Q$.

**The subresultant sequence can be computed using the method `sylvester_subresultant_sequence`.**

### Subsection 3.3 with formal examples

In this section we use all the previous described pieces to compute with the *spectral* operators:
$$L_u - \lambda\qquad\qquad\hat{P}_{2s+1} - \mu$$
where $\lambda, \mu$ are constants. We will then prepare the corresponding set up here and compute the objects described in Remark 3.8 and subsection 3.3.

In [16]:
SchR.<u,y> = RWOPolynomialRing(DifferentialRing(QQ["l","m"], lambda p : 0))
l,m = SchR.base().gens()
L = -y[2] + u[0]*y[0]

In [ ]:
%%time 
Phat = [Phat_odd(i,u,y,L) for i in range(10)] # computing the first Phat polynomials

##### **Some values for $\kdv$ and $v$**

At the beginning of subsection 3.3, the first values for $\kdv_2,\kdv_3$ and $v_2, v_3$ are written: 

In [17]:
show(r"\text{kdv}(2) \mapsto" + latex(kdv(2, u)))
show(r"\text{kdv}(3) \mapsto" + latex(kdv(3, u)))
show(r"v(2) \mapsto" + latex(v(2, u)))
show(r"v(3) \mapsto" + latex(v(3, u)))

\text{kdv}(2) \mapsto \frac{15}{8} u_{1} u_{0}^{2} + \left(-\frac{5}{8}\right) u_{3} u_{0} + \left(-\frac{5}{4}\right) u_{2} u_{1} + \frac{1}{16} u_{5}

\text{kdv}(3) \mapsto \frac{35}{16} u_{1} u_{0}^{3} + \left(-\frac{35}{32}\right) u_{3} u_{0}^{2} + \left(-\frac{35}{8}\right) u_{2} u_{1} u_{0} + \left(-\frac{35}{32}\right) u_{1}^{3} + \frac{7}{32} u_{5} u_{0} + \frac{21}{32} u_{4} u_{1} + \frac{35}{32} u_{3} u_{2} + \left(-\frac{1}{64}\right) u_{7}

v(2) \mapsto \frac{3}{8} u_{0}^{2} + \left(-\frac{1}{8}\right) u_{2}

v(3) \mapsto \frac{5}{16} u_{0}^{3} + \left(-\frac{5}{16}\right) u_{2} u_{0} + \left(-\frac{5}{32}\right) u_{1}^{2} + \frac{1}{32} u_{4}

##### **Some values for $P$**

Then, values for $P_3,P_5$ and $P_7$ are given:

In [18]:
show(r"P_3 \mapsto" + latex(P_odd(1,u,y,L)))
show(r"P_5 \mapsto" + latex(P_odd(2,u,y,L)))
show(r"P_7 \mapsto" + latex(P_odd(3,u,y,L)))

P_3 \mapsto \frac{3}{4} u_{1} y_{0} + \frac{3}{2} u_{0} y_{1} - y_{3}

P_5 \mapsto \frac{15}{8} u_{1} u_{0} y_{0} + \frac{15}{8} u_{0}^{2} y_{1} + \left(-\frac{15}{16}\right) u_{3} y_{0} + \left(-\frac{25}{8}\right) u_{2} y_{1} + \left(-\frac{15}{4}\right) u_{1} y_{2} + \left(-\frac{5}{2}\right) u_{0} y_{3} + 1 y_{5}

P_7 \mapsto \frac{105}{32} u_{1} u_{0}^{2} y_{0} + \frac{35}{16} u_{0}^{3} y_{1} + \left(-\frac{105}{32}\right) u_{3} u_{0} y_{0} + \left(-\frac{105}{16}\right) u_{2} u_{1} y_{0} + \left(-\frac{175}{16}\right) u_{2} u_{0} y_{1} + \left(-\frac{245}{32}\right) u_{1}^{2} y_{1} + \left(-\frac{105}{8}\right) u_{1} u_{0} y_{2} + \left(-\frac{35}{8}\right) u_{0}^{2} y_{3} + \frac{63}{64} u_{5} y_{0} + \frac{161}{32} u_{4} y_{1} + \frac{175}{16} u_{3} y_{2} + \frac{105}{8} u_{2} y_{3} + \frac{35}{4} u_{1} y_{4} + \frac{7}{2} u_{0} y_{5} - y_{7}

##### **Differential resultant of $L$ and $\hat{P}_3$**

Now we consider the Schrödinger operator $L_u = -\partial^2 + u$ and the related $\hat{P}_3(u,c^1)$ operator. We can observe that 
$$\hat{P}_3(u,c^1) = P_3 + c_1P_1.$$

In [19]:
Phat3 = Phat[1]; show(Phat3)

3/4*u_1*y_0 + 3/2*u_0*y_1 - y_3 + c_1*y_1

In [20]:
show(P_odd(0,u,y,L)); show(P_odd(1,u,y,L))

y_1

3/4*u_1*y_0 + 3/2*u_0*y_1 - y_3

We then proceed to compute the differential resultant of $L - \lambda$ and $\hat{P}_3 - \mu$:

In [21]:
%%time
R = Phat3.parent(); uR, yR = R.gens(); mR = R.base()("m"); lR = R.base()("l")
diff_res_L_Phat3 = R.sylvester_resultant(L - lR*yR[0], Phat3 - mR*yR[0], yR)
diff_res_L_Phat3

CPU times: user 103 ms, sys: 90 µs, total: 104 ms
Wall time: 103 ms


1/4*u_0^3 + (-1/8)*u_2*u_0 + 1/16*u_1^2 + (3/4*l + c_1)*u_0^2 + (-1/4*l - 1/4*c_1)*u_2 + (l*c_1 + c_1^2)*u_0 - l^3 - 2*l^2*c_1 - l*c_1^2 - m^2

In Remark 3.8, it was pointed out that this resultant can be written always as:
$$-\mu^2 + R_{2s+1}(u,c^s, \lambda),$$
where the $R_{2s+1}(u,c^s,\lambda)$ is a polynomial in $C\{u\}[c^s,\lambda]$. Let us showcase this:

In [22]:
flatten = diff_res_L_Phat3.flatten(); fR = flatten.parent(); fm = fR("m"); fl = fR("l")
show((flatten.degree(fm), [flatten.coefficient({fm:i}) for i in range(flatten.degree(fm)+1)]))
R3 = diff_res_L_Phat3.parent()(flatten.coefficient({fm:0}))
show(R3)

(2,
 [1/4*u_0^3 + 3/4*u_0^2*l - l^3 + u_0^2*c_1 + u_0*l*c_1 - 2*l^2*c_1 + u_0*c_1^2 - l*c_1^2 + 1/16*u_1^2 - 1/8*u_2*u_0 - 1/4*u_2*l - 1/4*u_2*c_1,
  0,
  -1])

1/4*u_0^3 + (-1/8)*u_2*u_0 + 1/16*u_1^2 + (3/4*l + c_1)*u_0^2 + (-1/4*l - 1/4*c_1)*u_2 + (l*c_1 + c_1^2)*u_0 - l^3 - 2*l^2*c_1 - l*c_1^2

This polynomial $R_3(u,c^1,\lambda)$ can be further splitted w.r.t. $\lambda$ with 
$$R_3(u,c^1, \lambda) = -\lambda^3 - 2c_1\lambda^2 + p_1(u,c^1)\lambda + p_0(u,c^1)$$

In [23]:
flatten_R3 = R3.flatten()
show((flatten_R3.degree(fl), [flatten_R3.coefficient({fl:i}) for i in range(flatten.degree(fl)+1)]))
p1 = diff_res_L_Phat3.parent()(flatten_R3.coefficient({fl:1}))
p0 = diff_res_L_Phat3.parent()(flatten_R3.coefficient({fl:0}))
show(r"p_0(u,c^1) = " + latex(p0))
show(r"p_1(u,c^1) = " + latex(p1))

(3,
 [1/4*u_0^3 + u_0^2*c_1 + u_0*c_1^2 + 1/16*u_1^2 - 1/8*u_2*u_0 - 1/4*u_2*c_1,
  3/4*u_0^2 + u_0*c_1 - c_1^2 - 1/4*u_2,
  -2*c_1,
  -1])

p_0(u,c^1) = \frac{1}{4} u_{0}^{3} + \left(-\frac{1}{8}\right) u_{2} u_{0} + \frac{1}{16} u_{1}^{2} + c_{1} u_{0}^{2} + \left(-\frac{1}{4} c_{1}\right) u_{2} + c_{1}^{2} u_{0}

p_1(u,c^1) = \frac{3}{4} u_{0}^{2} + \left(-\frac{1}{4}\right) u_{2} + c_{1} u_{0} - c_{1}^{2}

These two polynomials $p_0(u,c^1)$ and $p_1(u,c^1)$ are related with $\KdV_1(u,c^1)$:

In [24]:
p1.derivative() == KdV(1,u)

True

In [25]:
c_1 = KdV(1,u).parent()("c_1")
p0.derivative() == (u[0]*(1/2) + c_1)*KdV(1,u)

True

##### **Subresultant sequence of $L$ and $\hat{P}_3$**

Finally, the section 3.3 shows the formulas for the subresultant sequence between $L-\lambda$ and $\hat{P}_3 - \mu$:

In [28]:
%%time
diff_subresultant_seq = R.sylvester_subresultant_sequence(L - lR*yR[0], Phat3 - mR*yR[0], yR)
Lcal_0, Lcal_1 = diff_subresultant_seq

CPU times: user 9.96 ms, sys: 0 ns, total: 9.96 ms
Wall time: 9.71 ms


In [45]:
Lcal_0.coefficient(y[0]) == diff_res_L_Phat3

True

In [35]:
Lcal_1

(-1/4)*u_1*y_0 + 1/2*u_0*y_1 + (l + c_1)*y_1 + (-m)*y_0

In [37]:
show(r"\mathcal{L}_1[\partial] = " + latex(Lcal_1.coefficient(yR[1])))
show(r"\mathcal{L}_1[1] = " + latex(Lcal_1.coefficient(yR[0])))

\mathcal{L}_1[\partial] = \frac{1}{2} u_{0} + l + c_{1}

\mathcal{L}_1[1] = \left(-\frac{1}{4}\right) u_{1} - m

In [38]:
R.sylvester_matrix(L - lR*yR[0], Phat3 - mR*yR[0], yR, 1)

[      u_0 - l             0            -1             0]
[          u_1       u_0 - l             0            -1]
[  3/4*u_1 - m 3/2*u_0 + c_1             0            -1]